Below are parameters of the experiments reported in [KLNO24](https://eprint.iacr.org/2024/1972).

In [2]:
def largest_a_below_binom_sum(n, threshold_power):
    """
    Returns the largest a such that sum_{i=0}^a binomial(n, i) < 2^threshold_power

    Parameters:
    - n: total number of bits
    - threshold_power: the exponent k such that the threshold is 2^k

    Returns:
    - a: largest integer such that the cumulative binomial sum is less than 2^threshold_power
    """
    threshold = 2^threshold_power
    running_sum = 0
    a = 0

    while True:
        running_sum += binomial(n, a)
        if running_sum >= threshold:
            return a - 1
        a += 1

# Example usage:
assert largest_a_below_binom_sum(256, 128) == 29

In [120]:
from rok_estimator.rok_estimator import *


def RelationRPWithKJL(ratio = 18, height = 105):
    class JL:
        kjl = ratio
        njl = height
        mjl = kjl * njl
        alpha_inf = largest_a_below_binom_sum(mjl, mjl)
        def get_delta(self):
            return 2^(-self.njl)
    
        def get_right_bound(self):
            return sqrt(self.mjl * self.njl)
        def get_left_bound(self):
            return self.alpha_inf / (2 * sqrt(self.mjl))
    
        def get_right_bound_inf(self):
            return self.mjl
        def get_left_bound_inf(self):
            return self.alpha_inf / (2)
    
    
    class HJL: # heuristics from GHL, BS
        kjl = 34
        njl = 256
        mjl = kjl * njl
        alpha_inf = 29 
    
        def get_delta(self):
            return 2^(-128)
    
        def get_right_bound(self):
            return sqrt(337)
            
        def get_left_bound(self):
            return sqrt(30)
    
        def get_right_bound_inf(self):
            return self.mjl
        def get_left_bound_inf(self):
            return self.alpha_inf / (2)
    
    
    
    jl = JL()
    
    print("Limits of structure: ", jl.mjl)
    
    
    g_first_branch = None
    g_second_branch = None
    
    g_second_branch_ext_2_norm = None
    g_second_branch_ext_inf_norm = None
    
    class Relation_RP(Relation):
        def execute(self, op, **kwargs):
            match op:
                case "srp":
                    return self.pi_srp(**kwargs)
                case "to-first":
                    return self.pi_to_first(**kwargs)
                case "join":
                    return self.pi_join(**kwargs)
                case "norm-poly":
                    return self.pi_norm_poly()
                case _:
                    return super().execute(op, **kwargs)
                
        def pi_norm_poly(self):
            comm = (3 * ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.size_Rq()  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
            # comm = (2 * self.rep) * self.ring.size_Rq() + (1 * ceil(log(self.wdim, 2)) + 3 * self.rep) * self.ring.log_q  # TODO: Overestimating. Some of the communication are short elements. Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the communication cost.   
            snd_err = 3 * ceil(log(self.wdim, 2))/(2**(self.ring.log_q * self.ring.residue_deg)) # TODO: Assuming that wdim is a power of two for now. Even if not, this would be an upper bound of the soundness error.
            rel_params = {
                # "ring": self.ring,
                # "trivial": self.trivial,
                "op_name": "norm-poly",
                "n_compress": self.n_compress + 2,
                # "n_commit": self.n_commit,
                "n_rel": self.n_rel + 2,
                # "wdim": self.wdim,
                # "rep": self.rep,
                # "log_beta_wit_2": bound_log_canon_2_from_log_coeff_inf(self.ring,self.log_beta_wit_inf, dim=self.wdim * (self.rep + ell)), # Measured in Frobenius norm
                # "log_beta_wit_2": self.log_beta_wit_2, # Measured in max ell_2-norm over all columns
                # "log_beta_wit_inf": self.log_beta_wit_inf
                "comm" : comm,      
                "acc_comm" : self.acc_comm + comm,       
                "snd_err" : snd_err,
                "acc_snd_err" : self.acc_snd_err + snd_err,           
                "log_beta_ext_2_func" : lambda x : self.log_beta_wit_2, # perfect extraction
                "log_beta_ext_inf_func" : lambda x : oo # extraction of ell_2-norm is perfect, then bound ell_inf-norm by norm conversion
            }      
            return replace(self, **rel_params)
                
        def pi_srp(self):
            global g_first_branch 
            rel_params = {
                "n_rel": self.n_rel + 1,
                "log_slack_2_func": lambda x: x,
                "log_slack_inf_func": lambda x: x, 
            }
    
            g_first_branch = replace(deepcopy(self), **rel_params)
            comm = (self.n_commit + 1) * self.ring.size_Rq()  # for the new comm
            snd_err = self.wdim * self.rep / jl.kjl * (1 / 2**(self.ring.log_q * self.ring.residue_deg) + jl.get_delta() * euler_phi(self.ring.f) / jl.njl)
            global g_second_branch_ext_2_norm
            global g_second_branch_ext_inf_norm
    
            def log_beta_ext_2_func(x, _):
                global g_second_branch_ext_2_norm
                return g_second_branch_ext_2_norm + log(1 / jl.get_left_bound(), 2) + log(sqrt(radical(self.ring.f)),2)
                
            def log_beta_ext_inf_func(x, _):
                global g_second_branch_ext_inf_norm 
                return g_second_branch_ext_inf_norm + log(1 / jl.get_left_bound_inf(), 2)
                
            rel_params = {
                "op_name": "srp 2 rel",
                "n_compress": self.n_commit + 1,
                "n_rel": 1,
                "wdim": ZZ(ceil(self.wdim / jl.kjl * self.rep)),
                "rep": 1,
                "comm": comm,      
                "acc_comm" : self.acc_comm + comm,       
                "snd_err" : snd_err,
                "acc_snd_err" : self.acc_snd_err + snd_err,    
                "log_beta_wit_2": self.log_beta_wit_2 + log(jl.get_right_bound(), 2) + log(sqrt(self.rep),2),
                "log_beta_wit_inf":self.log_beta_wit_inf + log(jl.get_right_bound_inf(), 2) + log(sqrt(self.rep),2), # can we say sth about infinity - norm?
                "log_beta_ext_2_func": log_beta_ext_2_func, # we extract norm from the second branch (relax it)
                "log_beta_ext_inf_func" : log_beta_ext_inf_func,  # can we say sth about infinity - norm?
                "log_slack_2_func": lambda x: 0,
                "log_slack_inf_func": lambda x: 0,
            }     
            
            # We immediately switch to the second branch to estimate binding and serve as an extraction checkpoint. 
            return replace(self, **rel_params)
    
    
        def pi_to_first(self):
            global g_second_branch 
            g_second_branch = deepcopy(self)
            
            rel_params = {
                "op_name": "    1 rel",
                "n_rel": 1,
                "log_beta_ext_2_func" : lambda x, _ : x, # perfect extraction
                "log_beta_ext_inf_func" : lambda x, _ : x, # perfect extraction
                "acc_comm": g_second_branch.acc_comm,
                "comm": 0,
                "snd_err" : g_second_branch.snd_err,
                "acc_snd_err" : g_second_branch.acc_snd_err,    
                # "log_slack_2_func" : lambda x: 0, # perfect extraction
                # "log_slack_inf_func" : lambda x: 0 # perfect extraction
            }      
            global g_first_branch
            return replace(g_first_branch, **rel_params)
            
        def pi_join(self):
            def log_beta_ext_2_func(x, _):
                global g_second_branch_ext_2_norm
                g_second_branch_ext_2_norm = x
                return x
            def log_beta_ext_inf_func(x, _):
                global g_second_branch_ext_inf_norm
                g_second_branch_ext_inf_norm = x
                return x
    
            global g_second_branch
    
            assert g_second_branch.wdim <= self.wdim
            assert g_second_branch.rep == 1
            assert self.rep == 1
            
            rel_params = {
                "op_name": "join",
                "rep": 2,
                "n_rel": self.n_rel + g_second_branch.n_rel,
                "log_beta_wit_2": max(self.log_beta_wit_2, g_second_branch.log_beta_wit_2),
                "log_beta_wit_inf": max(self.log_beta_wit_inf, g_second_branch.log_beta_wit_inf),
                "log_beta_ext_2_func": log_beta_ext_2_func, # perfect extraction, but we keep track of what was there
                "log_beta_ext_inf_func" : log_beta_ext_inf_func # perfect extraction, but we keep track of what was there
            }      
            return replace(self, **rel_params)
    return Relation_RP
        

def challenge_set(max_v = 100,  f = 60):
    K = CyclotomicField(f)
    phi = euler_phi(f)
    R = K.ring_of_integers()
    
    
    return ChallengeSet(cardinality = max_v^phi, gamma_2 = max_v * sqrt(phi) / 2, theta_2 = max_v * sqrt(phi), gamma_inf = max_v / 2, theta_inf = max_v)

In [27]:
# A

ring_params = {
    "f": 60,
    "log_beta_sis_2": 37.8,
    "log_q": 64,
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = [("split", {"d": 25})] + 12 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 33 | 37/  0 )        |     ( 14 | 34/  0 )         | 7688. MB   |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 split      |   10747904 |  25 |    ( 31 | 34/  0 )        |     ( 14 | 34/  0 )         | 7688. MB   |      (165.0 KB | 165.0 KB)      |         (2^-123 | 2^-123)         
 bdecomp    |   10747904 | 100 |    ( 19 | 23/  0 )        |     (  3 | 23/  0 )         | 8200. MB   |      (515.6 KB | 680.6 KB)      |         (2^-oo  | 2^-123)         
 norm       |   10747904 | 109 |    ( 19 | 37/  0 )        |     (  3 | 37/  0 )         | 8938. MB   |      (102.8 KB | 783.4 KB)      |         (2^-124 | 2^-122)         
 batch      |   10747904 | 109 |    ( 19 | 37/  0 )        |     (  3 | 37/  0 )         | 8938. MB   |      (0.0000 B

In [30]:
# A NEW

ring_params = {
    "f": 60,
    "log_beta_sis_2": 72.2,
    "log_q": 74,
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("join", {})
])

ops = loop(2) + loop(2) * 14+ [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(18, 105))
sim.ring.C = challenge_set(100)


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  1890
Ring parameters:
    conductor f: 60, degree phi: 16, modulus q: 2^74, beta_sis_2: 2^72.2000000000000
    SIS module rank n_sis: 171, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 148
    |R_q| = 148.0 B, |R_q^(n_sis)| = 24.71 KB
 
Challenge set parameters:
    cardinality: 100000000000000000000000000000000
    forward ell_2 expansion factor gamma_2: 2^8
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^6
    inverse ell_inf expansion factor theta_inf: 2^7
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 33 | 33/  0 )        |     ( 14 | 33/  0 )         | 7688. MB   |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )       

In [165]:
# A 256

ring_params = {
    "f": 60 * 2,
    "log_beta_sis_2": 45.8,
    "log_q": 64,
    "secpar_target": 256,
    "kappa_target": 160
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = [("split", {"d": 25})] + 12 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 34 | 38/  0 )        |     ( 14 | 35/  0 )         | 15380. MB  |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 split      |   10747904 |  25 |    ( 32 | 35/  0 )        |     ( 14 | 35/  0 )         | 15380. MB  |      (444.0 KB | 444.0 KB)      |         (2^-187 | 2^-187)         
 bdecomp    |   10747904 | 100 |    ( 20 | 24/  0 )        |     (  3 | 24/  0 )         | 16400. MB  |      (1.355 MB | 1.789 MB)      |         (2^-oo  | 2^-187)         
 norm       |   10747904 | 109 |    ( 20 | 44/  0 )        |     (  3 | 43/  0 )         | 17880. MB  |      (248.2 KB | 2.031 MB)      |         (2^-188 | 2^-186)         
 batch      |   10747904 | 109 |    ( 20 | 44/  0 )        |     (  3 | 43/  0 )         | 17880. MB  |      (0.0000 B

In [163]:
# A NEW 256

ring_params = {
    "f": 60 * 2,
    "log_beta_sis_2": 82.8,
    "log_q": 84.5,
    "secpar_target": 256,
    "kappa_target": 160
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("join", {})
])

ops = loop(2) + loop(2) * 14+ [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(18, 105))
sim.ring.C = challenge_set(639, ring_params["f"])


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  1890
Ring parameters:
    conductor f: 120, degree phi: 32, modulus q: 2^84.5000000000000, beta_sis_2: 2^82.8000000000000
    SIS module rank n_sis: 181, target SIS security: 256, resulting SIS security: 256
    residue degree: 2, target Schwartz-Zippel security: 160, resulting Schwartz-Zippel security: 169.000000000000
    |R_q| = 338.0 B, |R_q^(n_sis)| = 59.74 KB
 
Challenge set parameters:
    cardinality: 597073039182879857989553168123669664326517795218001082090332702878587253341837810932101121
    forward ell_2 expansion factor gamma_2: 2^11
    inverse ell_2 expansion factor theta_2: 2^12
    forward ell_inf expansion factor gamma_inf: 2^9
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 34 | 34/  0 )        |     ( 14 | 34/  0 ) 

In [144]:
# A 512

ring_params = {
    "f": 60 * 4,
    "log_beta_sis_2": 49.8,
    "log_q": 64,
    "secpar_target": 512,
    "kappa_target": 320
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = [("split", {"d": 25})] + 12 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  268697600 |   1 |    ( 35 | 39/  0 )        |     ( 14 | 36/  0 )         | 30750. MB  |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 split      |   10747904 |  25 |    ( 33 | 36/  0 )        |     ( 14 | 36/  0 )         | 30750. MB  |      (936.0 KB | 936.0 KB)      |         (2^-379 | 2^-379)         
 bdecomp    |   10747904 | 100 |    ( 21 | 25/  0 )        |     (  3 | 25/  0 )         | 32800. MB  |      (2.856 MB | 3.771 MB)      |         (2^-oo  | 2^-379)         
 norm       |   10747904 | 109 |    ( 21 | 47/  0 )        |     (  3 | 47/  0 )         | 35750. MB  |      (514.5 KB | 4.273 MB)      |         (2^-380 | 2^-378)         
 batch      |   10747904 | 109 |    ( 21 | 47/  0 )        |     (  3 | 47/  0 )         | 35750. MB  |      (0.0000 B

In [142]:
# A NEW 512

ring_params = {
    "f": 60 * 4,
    "log_beta_sis_2": 88.8,
    "log_q": 90.5,
    "secpar_target": 512,
    "kappa_target": 320
}

rel_params = {
    "wdim": 2**18 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("join", {})
])

ops = loop(2) + loop(2) * 14+ [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(18, 105))
sim.ring.C = challenge_set(1015, ring_params["f"])


sim.execute(ops)
print(sim.ring)
sim.extract()
sim.show()

Limits of structure:  1890
Ring parameters:
    conductor f: 240, degree phi: 64, modulus q: 2^90.5000000000000, beta_sis_2: 2^88.8000000000000
    SIS module rank n_sis: 175, target SIS security: 512, resulting SIS security: 514
    residue degree: 4, target Schwartz-Zippel security: 320, resulting Schwartz-Zippel security: 362.000000000000
    |R_q| = 724.0 B, |R_q^(n_sis)| = 123.7 KB
 
Challenge set parameters:
    cardinality: 2593144416083281573937354089607960550974406392323857693948973123764136144062028580931616790043839060271748341128957793735418248534166008463561125391692342712079977218309068121016025543212890625
    forward ell_2 expansion factor gamma_2: 2^12
    inverse ell_2 expansion factor theta_2: 2^13
    forward ell_inf expansion factor gamma_inf: 2^9
    inverse ell_inf expansion factor theta_inf: 2^10
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness err

In [6]:
# B NEW

ring_params = {
    "f": 60,
    "log_beta_sis_2": 73.6,
    "log_q": 75.6,
}

rel_params = {
    "wdim": 2**20 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("join", {})
])

ops = loop(2) + loop(2) * 16 + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(20, 108))
sim.ring.C = C

print(sim.ring)
sim.execute(ops)
sim.extract()
sim.show()

Limits of structure:  2160
Ring parameters:
    conductor f: 60, degree phi: 16, modulus q: 2^75.6000000000000, beta_sis_2: 2^73.6000000000000
    SIS module rank n_sis: 174, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 151.200000000000
    |R_q| = 151.2 B, |R_q^(n_sis)| = 25.69 KB
 
Challenge set parameters:
    cardinality: 100000000000000000000000000000000
    forward ell_2 expansion factor gamma_2: 2^8
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^6
    inverse ell_inf expansion factor theta_inf: 2^7
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       | 1074790400 |   1 |    ( 34 | 34/  0 )        |     ( 14 | 34/  0 )         | 30750. MB  |      (0.0000 B | 0.0000 B)      |      

In [105]:
# C

ring_params = {
    "f": 60,
    "log_beta_sis_2": 39.8,
    "log_q": 64,
}

rel_params = {
    "wdim": 2**22 * 41,
    "rep": 25,
    "log_beta_wit_inf": 14,
}

ops_params = {
    "ell": 4,
    "d": 2,
}

loop = [
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("norm", {}), 
    ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold_old", {})
]
ops = 16 * loop + [("finish", {})]

sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()


Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       |  171966464 |  25 |    ( 33 | 36/  0 )        |     ( 14 | 36/  0 )         | 123000. MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 bdecomp    |  171966464 | 100 |    ( 21 | 25/  0 )        |     (  3 | 25/  0 )         | 131200. MB |      (571.9 KB | 571.9 KB)      |         (2^-oo  | 2^-oo )         
 norm       |  171966464 | 110 |    ( 21 | 39/  0 )        |     (  3 | 39/  0 )         | 144300. MB |      (117.5 KB | 689.4 KB)      |         (2^-124 | 2^-124)         
 batch      |  171966464 | 110 |    ( 21 | 39/  0 )        |     (  3 | 39/  0 )         | 144300. MB |      (0.0000 B | 689.4 KB)      |         (2^-119 | 2^-119)         
 split      |   85983232 | 220 |    ( 21 | 39/  0 )        |     (  3 | 39/  0 )         | 144300. MB |      (880.0 KB

In [7]:
# C NEW

ring_params = {
    "f": 60,
    "log_beta_sis_2": 74.7,
    "log_q": 76.9,
}

rel_params = {
    "wdim": 2**22 * 41 * 25,
    "rep": 1,
    "log_beta_wit_inf": 14
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = lambda ell: ([
    ("bdecomp", {"ell":ell }), 
    ("norm", {}), 
    ("batch", {}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("join", {})
])

ops = loop(2) + loop(2) * 18 + [("finish", {})]

sim = Simulation(ring_params, rel_params, RelationRPWithKJL(20, 110))
sim.ring.C = C


print(sim.ring)
sim.execute(ops)
sim.extract()
sim.show()

Limits of structure:  2200
Ring parameters:
    conductor f: 60, degree phi: 16, modulus q: 2^76.9000000000000, beta_sis_2: 2^74.7000000000000
    SIS module rank n_sis: 176, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 153.800000000000
    |R_q| = 153.8 B, |R_q^(n_sis)| = 26.43 KB
 
Challenge set parameters:
    cardinality: 100000000000000000000000000000000
    forward ell_2 expansion factor gamma_2: 2^8
    inverse ell_2 expansion factor theta_2: 2^9
    forward ell_inf expansion factor gamma_inf: 2^6
    inverse ell_inf expansion factor theta_inf: 2^7
Execution Trace:
 operation  |    wdim    | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) |  wit size  | communication  (growth | total) | soundness error  (growth | total) 
 init       | 4299161600 |   1 |    ( 35 | 35/  0 )        |     ( 14 | 35/  0 )         | 123000. MB |      (0.0000 B | 0.0000 B)      |      